In [1]:
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [32]:
# Q1 count of columns
df_train = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')


df_train.shape

(3066766, 19)

In [34]:
# Q2. Computing duration
df_train['duration'] = df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']
df_train['duration'] = df_train.duration.apply(lambda td: td.total_seconds()/60)
df_train['duration'].std() # 42.59

np.float64(42.59435124195458)

In [38]:
# Q3. Dropping outliers

# Total number of records before filtering
total_records = len(df_train)

# Filter records with duration between 1 and 60 minutes (inclusive)
filtered_df = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)]

# Number of records after filtering
filtered_records = len(filtered_df)

# Calculate fraction of records left
fraction_left = filtered_records / total_records

print(f"Fraction of records left after removing outliers: {fraction_left:.4f}")
# filtered_df.head()


Fraction of records left after removing outliers: 0.9812


In [40]:
# Q4. One-hot encoding

# Convert IDs to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Keep only relevant columns
df_subset = filtered_df[['PULocationID', 'DOLocationID']]

dv = DictVectorizer()
train_dicts = df_subset.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts) 
X_train # 525


/var/folders/nt/c007l6_s6xjgg3s2fzgqgfy40000gn/T/ipykernel_15079/2208270426.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
/var/folders/nt/c007l6_s6xjgg3s2fzgqgfy40000gn/T/ipykernel_15079/2208270426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>

In [48]:
# Q5. Training a model
target = 'duration'
y_train = filtered_df[target].values
# print(y_train)
# print(X_train)
import numpy as np

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

# mean_squared_error(y_train, y_pred) # 1763.70
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(rmse)

7.649262183753913


In [50]:
# Q6. Evaluating the model

df_feb = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration'] = df_feb.duration.apply(lambda td: td.total_seconds()/60)
df_feb['duration'].std() # 42.59

filtered_df_feb = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

# Convert IDs to strings
filtered_df_feb['PULocationID'] = filtered_df_feb['PULocationID'].astype(str)
filtered_df_feb['DOLocationID'] = filtered_df_feb['DOLocationID'].astype(str)

# Keep only relevant columns
df_subset_feb = filtered_df_feb[['PULocationID', 'DOLocationID']]

dv = DictVectorizer()
train_dicts = df_subset_feb.to_dict(orient='records')
X_train_feb = dv.fit_transform(train_dicts) 
X_train_feb # 525

target = 'duration'
y_train_feb = filtered_df_feb[target].values

lr_feb = LinearRegression()
lr_feb.fit(X_train_feb, y_train_feb)
y_pred_feb = lr_feb.predict(X_train_feb)

# mean_squared_error(y_train, y_pred) # 1763.70
rmse_feb = np.sqrt(mean_squared_error(y_train_feb, y_pred_feb))
print(rmse_feb)

/var/folders/nt/c007l6_s6xjgg3s2fzgqgfy40000gn/T/ipykernel_15079/2056162970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_feb['PULocationID'] = filtered_df_feb['PULocationID'].astype(str)
/var/folders/nt/c007l6_s6xjgg3s2fzgqgfy40000gn/T/ipykernel_15079/2056162970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_feb['DOLocationID'] = filtered_df_feb['DOLocationID'].astype(str)


7.778948575332285
